In [1]:
# Importing required libraries 
# Keras
from keras.models import model_from_json

# Other  
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import seaborn as sns
import os
import pickle
import wave
import IPython.display as ipd  # To play sound in the notebook
import python_speech_features as ps
import seaborn as sns
import scipy
import math
import time
from scipy import signal
import sys

In [2]:
def trimVoice(data, sokuonleng = 3, spaceleng = 10, voiceleng= 10, debug=False):
    trimpoint = detectVoice(data, sokuonleng = sokuonleng, spaceleng=spaceleng, voiceleng = voiceleng, debug=debug)
    trimdata = []
    for point in trimpoint:
        trimdata.append(data[point[0]:point[1]])
    return trimdata

In [3]:
def detectVoice(data, sokuonleng = 3, spaceleng = 10, voiceleng= 10, debug=False):
    zscore = np.empty_like(data)
    eps = 1e-5
    mean = np.mean(data)
    std = np.std(data)
    zscore = (data - mean)/(std+eps)
    zscore = np.append(zscore, np.zeros(1024 - zscore.shape[0] % 1024),0)
    zscore = zscore.reshape(-1, 1024)
    ave = np.empty((1,), float)
    diff = np.empty((0,), float)
    silent = 0
    voice = 0
    isVoice = 0
    standby = 0
    voiceOnWave = np.empty((0,2), int)
    warn = False
    for waddr in range(zscore.shape[0]):
        ave = np.append(ave, np.mean(np.abs(zscore[waddr])))
        diff = np.append(diff, ave[-2]-ave[-1])
        #print(np.abs(diff[waddr]) > 0)
        #声の部分か判定する。
        if np.abs(diff[waddr]) > 0.05:
            silent = 0
            voice  = sokuonleng
        else:
            silent += 1
            voice  -= 1 if voice > 0 else 0
        #発話していないなら
        if isVoice == 0:
            if standby == -1:
                #スタンバイでない時、音を検知したら、スタンバイする(その時の時間を覚えておく)。
                if voice == sokuonleng:
                    standby = waddr - 1 #発声する瞬間も大事な特微量なので、現在のアドレスより1コマ過去のものから判定開始
            else:
                if waddr-standby >= voiceleng:
                    #voiceleng以上発話していれば、isRecをアクティブに。
                    if voice != 0:
                        if debug == True:
                            print("Record start{}".format(standby, waddr))
                        isVoice = 1
                        voiceOnWave = np.append(voiceOnWave, np.zeros((1,2), int),0)
                        voiceOnWave[-1,0] = standby*1024
                    #voicelengより短い発話だったら、スタンバイ解除
                    elif voice == 0:
                        standby = -1
                elif silent >= spaceleng:
                    standby = -1
                
        #発話しているなら
        else:
            #もしも、一定時間以上静寂なら、isRecをディスアクティブに。
            if silent >= spaceleng or waddr == zscore.shape[0]-1:
                if debug == True:
                    print("--->{}".format(waddr-spaceleng, waddr))
                isVoice = 0
                standby = -1
                voiceOnWave[-1,1] = (waddr - silent +5)*1024 #+5して発話後に若干余白をとる。
    return voiceOnWave

In [4]:
def loadwav(path, mono=True):
    file = wave.open(path, 'r')
    params = file.getparams()
    nchannels, samplewidth, samplerate, wav_length = params[:4]
    datatype = {2:"int16", 4:"int32"}
    str_data = file.readframes(wav_length)
    data = np.frombuffer(str_data, dtype = np.short)
    if(nchannels == 2):
        left = data[::2]
        right= data[1::2]
        wavedata = np.stack([left, right], 1)
        if mono == True:
            wavedata = np.mean(wavedata, axis=1)
            wavedata = wavedata.astype(datatype[samplewidth])
    else:
        wavedata = data
    return wavedata, samplerate, samplewidth, nchannels

In [5]:
def savewav(data, path):
    outd = struct.pack("h" * len(data), *data)
    with wave.open(path, 'w') as ww:
        ww.setnchannels(nchannel)
        ww.setsampwidth(samplewidt)
        ww.setframerate(samplerate)
        ww.writeframes(outd)

In [6]:
def mellCepstrum(data, RATE, winlen=0.08, winstep=0.016, nfilt=40, start=None, end=None):
    data = np.append(data, np.zeros(1024 - data.shape[0] % 1024),0)
    eps = 1e-5
    if start == None and end == None:
        mel_spec = ps.logfbank(data, samplerate=RATE, winlen=winlen, winstep=winstep, nfilt=nfilt, nfft=int(winlen*RATE))
    elif start == None or end == None:
        point = start if end == None else end
        mel_spec = ps.logfbank(data[point], samplerate=RATE, winlen=winlen, winstep=winstep, nfilt=nfilt, nfft=int(winlen*RATE))
    else:
        mel_spec = ps.logfbank(data[start:end], samplerate=RATE, winlen=winlen, winstep=winstep, nfilt=nfilt, nfft=int(winlen*RATE))
    #deltaの特微量を生成
    delta1= ps.delta(mel_spec, 4)
    #delta-deltaの特微量を生成
    delta2 = ps.delta(delta1, 4)
    mean1 = np.mean(mel_spec)
    mean2 = np.mean(delta1)
    mean3 = np.mean(delta2)
    std1 = np.std(mel_spec)
    std2 = np.std(delta1)
    std3 = np.std(delta2)
    procd = np.empty((mel_spec.shape[0], nfilt, 3))
    procd[:,:,0] = (mel_spec - mean1)/(std1+eps) #mel_spec
    procd[:,:,1] = (delta1 - mean2)/(std2+eps) #delta1
    procd[:,:,2] = (delta2 - mean3)/(std3+eps) #delta2
    return procd

In [7]:
def convlabel(label):
    tag = {'angry':0, 'disgust':1, 'fear':2, 'happy':3,
 'neutral':4, 'sad':5, 'surprise':6}
    return tag[label]

In [8]:
def inverselabel(value):
    tag = {'angry':0, 'disgust':1, 'fear':2, 'happy':3,
 'neutral':4, 'sad':5, 'surprise':6}
    return [k for k, v in tag.items() if v == value][0]

In [9]:
# loading json and model architecture 
json_file = open('model_json.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("saved_models/Emotion_Model.h5")
print("Loaded model from disk")

Loaded model from disk


In [10]:
# lets pick up the meta-data that we got from our first part of the Kernel
ref = pd.read_csv("../2020_12_12_SERv3/Data_path.csv")
ref.head()

,labels,source,path
0,happy,SAVEE,/home/naokitakatani/datasets/SAVEE/DC_h03.wav
1,fear,SAVEE,/home/naokitakatani/datasets/SAVEE/JE_f12.wav
2,happy,SAVEE,/home/naokitakatani/datasets/SAVEE/DC_h02.wav
3,neutral,SAVEE,/home/naokitakatani/datasets/SAVEE/KL_n01.wav
4,happy,SAVEE,/home/naokitakatani/datasets/SAVEE/DC_h07.wav


In [11]:
features = np.empty((0,300,40,3),float)
emolabel = np.zeros(0,int)
setindex = np.zeros(0,int)
start = 0
b, a = signal.iirfilter(2, 1500,btype='highpass',rs = 60,rp = 1.0, ftype='ellip',output='ba',fs=44100)

In [13]:
for index, (label, path) in enumerate(zip(ref.labels, ref.path)):
    if index == start: 
        if features.shape[0] == index:
            features = np.append(features,np.zeros((1,300,40,3),float),0)
        elif features.shape[0] != index + 1:
            sys.exit()
        print("No.{} path : {}".format(index, path))
        try:
            X, samplerate,samplewidth,nchannel = loadwav(path)
            trimdata = trimVoice(X, spaceleng = 30)
            Y = trimdata[0]
            heatmap_data = mellCepstrum(Y, samplerate)
            end = heatmap_data.shape[0] if heatmap_data.shape[0] <= 300 else 300
            features[-1,:end] = heatmap_data[:end]
        except KeyboardInterrupt:
            print("KeyboardInterrupt")
            sys.exit()
        #except:
        #    print("Skip this audio file.")
        start += 1

No.92 path : /home/naokitakatani/datasets/SAVEE/DC_f11.wav


FileNotFoundError: [Errno 2] No such file or directory: '/home/naokitakatani/datasets/SAVEE/DC_f11.wav'

In [23]:
preds = loaded_model.predict(features, 
                         batch_size=16, 
                         verbose=1)
#13の感情のなかから最も高い値(確率が高い感情)を選ぶ
preds=preds.argmax(axis=1)

761/761 [==============================] - 2177s 3s/step


In [24]:
with open("nonFilterFeatures.pkl", "wb") as file:
    pickle.dump((features, preds), file)

In [17]:
with open("nonFilterFeatures-preds.pkl", "wb") as file:
    pickle.dump(preds, file)

In [10]:
features = np.empty((0,300,40,3),float)
emolabel = np.zeros(0,int)
setindex = np.zeros(0,int)
start = 0
b, a = signal.iirfilter(2, 1500,btype='highpass',rs = 60,rp = 1.0, ftype='ellip',output='ba',fs=44100)

In [12]:
for index, (label, path) in enumerate(zip(ref.labels, ref.path)):
    if index == start: 
        if features.shape[0] == index:
            features = np.append(features,np.zeros((1,300,40,3),float),0)
        elif features.shape[0] != index + 1:
            sys.exit()
        print("No.{} path : {}".format(index, path))
        try:
            X, samplerate,samplewidth,nchannel = loadwav(path)
            trimdata = trimVoice(X, spaceleng = 30)
            Y = signal.lfilter(b, a, trimdata[0])
            heatmap_data = mellCepstrum(Y, samplerate)
            end = heatmap_data.shape[0] if heatmap_data.shape[0] <= 300 else 300
            features[-1,:end] = heatmap_data[:end]
        except KeyboardInterrupt:
            print("KeyboardInterrupt")
            sys.exit()
        except:
            print("Skip this audio file.")
        start += 1

No.0 path : /home/naokitakatani/datasets/SAVEE/DC_h03.wav
No.1 path : /home/naokitakatani/datasets/SAVEE/JE_f12.wav
No.2 path : /home/naokitakatani/datasets/SAVEE/DC_h02.wav
No.3 path : /home/naokitakatani/datasets/SAVEE/KL_n01.wav
No.4 path : /home/naokitakatani/datasets/SAVEE/DC_h07.wav
No.5 path : /home/naokitakatani/datasets/SAVEE/JE_h14.wav
No.6 path : /home/naokitakatani/datasets/SAVEE/DC_su08.wav
No.7 path : /home/naokitakatani/datasets/SAVEE/DC_n19.wav
No.8 path : /home/naokitakatani/datasets/SAVEE/DC_a09.wav
No.9 path : /home/naokitakatani/datasets/SAVEE/JE_h05.wav
No.10 path : /home/naokitakatani/datasets/SAVEE/JK_sa01.wav
No.11 path : /home/naokitakatani/datasets/SAVEE/DC_a05.wav
No.12 path : /home/naokitakatani/datasets/SAVEE/DC_su13.wav
No.13 path : /home/naokitakatani/datasets/SAVEE/DC_d02.wav
No.14 path : /home/naokitakatani/datasets/SAVEE/DC_sa04.wav
No.15 path : /home/naokitakatani/datasets/SAVEE/JK_h06.wav
No.16 path : /home/naokitakatani/datasets/SAVEE/JK_f06.wav
No.

No.142 path : /home/naokitakatani/datasets/SAVEE/DC_f08.wav
No.143 path : /home/naokitakatani/datasets/SAVEE/KL_d01.wav
No.144 path : /home/naokitakatani/datasets/SAVEE/JK_f13.wav
No.145 path : /home/naokitakatani/datasets/SAVEE/DC_f09.wav
No.146 path : /home/naokitakatani/datasets/SAVEE/DC_a04.wav
No.147 path : /home/naokitakatani/datasets/SAVEE/KL_sa15.wav
No.148 path : /home/naokitakatani/datasets/SAVEE/KL_a06.wav
No.149 path : /home/naokitakatani/datasets/SAVEE/DC_sa03.wav
No.150 path : /home/naokitakatani/datasets/SAVEE/JE_d04.wav
No.151 path : /home/naokitakatani/datasets/SAVEE/KL_h13.wav
No.152 path : /home/naokitakatani/datasets/SAVEE/JK_h15.wav
No.153 path : /home/naokitakatani/datasets/SAVEE/JK_h14.wav
No.154 path : /home/naokitakatani/datasets/SAVEE/JK_su04.wav
No.155 path : /home/naokitakatani/datasets/SAVEE/JE_a15.wav
No.156 path : /home/naokitakatani/datasets/SAVEE/KL_a09.wav
No.157 path : /home/naokitakatani/datasets/SAVEE/JE_sa13.wav
No.158 path : /home/naokitakatani/da

No.282 path : /home/naokitakatani/datasets/SAVEE/JE_a10.wav
No.283 path : /home/naokitakatani/datasets/SAVEE/JE_sa11.wav
No.284 path : /home/naokitakatani/datasets/SAVEE/JK_su05.wav
No.285 path : /home/naokitakatani/datasets/SAVEE/DC_a12.wav
No.286 path : /home/naokitakatani/datasets/SAVEE/KL_n29.wav
No.287 path : /home/naokitakatani/datasets/SAVEE/JK_h13.wav
No.288 path : /home/naokitakatani/datasets/SAVEE/KL_h02.wav
No.289 path : /home/naokitakatani/datasets/SAVEE/DC_a10.wav
No.290 path : /home/naokitakatani/datasets/SAVEE/JK_n20.wav
No.291 path : /home/naokitakatani/datasets/SAVEE/JE_n17.wav
No.292 path : /home/naokitakatani/datasets/SAVEE/JE_n13.wav
No.293 path : /home/naokitakatani/datasets/SAVEE/JK_n25.wav
No.294 path : /home/naokitakatani/datasets/SAVEE/KL_a13.wav
No.295 path : /home/naokitakatani/datasets/SAVEE/JE_n08.wav
No.296 path : /home/naokitakatani/datasets/SAVEE/JE_n21.wav
No.297 path : /home/naokitakatani/datasets/SAVEE/KL_a11.wav
No.298 path : /home/naokitakatani/data

No.423 path : /home/naokitakatani/datasets/SAVEE/DC_n21.wav
No.424 path : /home/naokitakatani/datasets/SAVEE/DC_sa14.wav
No.425 path : /home/naokitakatani/datasets/SAVEE/DC_n15.wav
No.426 path : /home/naokitakatani/datasets/SAVEE/JK_n05.wav
No.427 path : /home/naokitakatani/datasets/SAVEE/JE_n04.wav
No.428 path : /home/naokitakatani/datasets/SAVEE/JK_sa06.wav
No.429 path : /home/naokitakatani/datasets/SAVEE/JK_d13.wav
No.430 path : /home/naokitakatani/datasets/SAVEE/DC_n10.wav
No.431 path : /home/naokitakatani/datasets/SAVEE/KL_f13.wav
No.432 path : /home/naokitakatani/datasets/SAVEE/DC_h05.wav
No.433 path : /home/naokitakatani/datasets/SAVEE/JK_a01.wav
No.434 path : /home/naokitakatani/datasets/SAVEE/JK_sa11.wav
No.435 path : /home/naokitakatani/datasets/SAVEE/KL_n13.wav
No.436 path : /home/naokitakatani/datasets/SAVEE/JK_h04.wav
No.437 path : /home/naokitakatani/datasets/SAVEE/JE_h04.wav
No.438 path : /home/naokitakatani/datasets/SAVEE/KL_n09.wav
No.439 path : /home/naokitakatani/dat

No.537 path : /home/naokitakatani/datasets/RAVDESS/Actor_01/03-01-08-02-02-02-01.wav
No.538 path : /home/naokitakatani/datasets/RAVDESS/Actor_01/03-01-03-01-02-01-01.wav
No.539 path : /home/naokitakatani/datasets/RAVDESS/Actor_01/03-01-03-02-02-01-01.wav
No.540 path : /home/naokitakatani/datasets/RAVDESS/Actor_02/03-01-07-01-01-02-02.wav
No.541 path : /home/naokitakatani/datasets/RAVDESS/Actor_02/03-01-06-02-02-02-02.wav
No.542 path : /home/naokitakatani/datasets/RAVDESS/Actor_02/03-01-04-01-02-02-02.wav
No.543 path : /home/naokitakatani/datasets/RAVDESS/Actor_02/03-01-05-02-01-02-02.wav
No.544 path : /home/naokitakatani/datasets/RAVDESS/Actor_02/03-01-02-01-01-02-02.wav
No.545 path : /home/naokitakatani/datasets/RAVDESS/Actor_02/03-01-01-01-01-02-02.wav
No.546 path : /home/naokitakatani/datasets/RAVDESS/Actor_02/03-01-05-01-02-02-02.wav
No.547 path : /home/naokitakatani/datasets/RAVDESS/Actor_02/03-01-05-01-01-01-02.wav
No.548 path : /home/naokitakatani/datasets/RAVDESS/Actor_02/03-01

...

No.11634 path : /home/naokitakatani/datasets/CREMA-D/1085_IWW_ANG_XX.wav
No.11635 path : /home/naokitakatani/datasets/CREMA-D/1085_IWW_DIS_XX.wav
No.11636 path : /home/naokitakatani/datasets/CREMA-D/1085_IWW_FEA_XX.wav
No.11637 path : /home/naokitakatani/datasets/CREMA-D/1085_IWW_HAP_XX.wav
No.11638 path : /home/naokitakatani/datasets/CREMA-D/1085_IWW_NEU_XX.wav
No.11639 path : /home/naokitakatani/datasets/CREMA-D/1085_IWW_SAD_XX.wav
No.11640 path : /home/naokitakatani/datasets/CREMA-D/1085_MTI_ANG_XX.wav
No.11641 path : /home/naokitakatani/datasets/CREMA-D/1085_MTI_DIS_XX.wav
No.11642 path : /home/naokitakatani/datasets/CREMA-D/1085_MTI_FEA_XX.wav
No.11643 path : /home/naokitakatani/datasets/CREMA-D/1085_MTI_HAP_XX.wav
No.11644 path : /home/naokitakatani/datasets/CREMA-D/1085_MTI_NEU_XX.wav
No.11645 path : /home/naokitakatani/datasets/CREMA-D/1085_MTI_SAD_XX.wav
No.11646 path : /home/naokitakatani/datasets/CREMA-D/1085_TAI_ANG_XX.wav
No.11647 path : /home/naokitakatani/datasets/CREMA-

No.11747 path : /home/naokitakatani/datasets/CREMA-D/1086_WSI_DIS_XX.wav
No.11748 path : /home/naokitakatani/datasets/CREMA-D/1086_WSI_FEA_XX.wav
No.11749 path : /home/naokitakatani/datasets/CREMA-D/1086_WSI_HAP_XX.wav
No.11750 path : /home/naokitakatani/datasets/CREMA-D/1086_WSI_NEU_XX.wav
No.11751 path : /home/naokitakatani/datasets/CREMA-D/1086_WSI_SAD_XX.wav
No.11752 path : /home/naokitakatani/datasets/CREMA-D/1087_DFA_ANG_XX.wav
No.11753 path : /home/naokitakatani/datasets/CREMA-D/1087_DFA_DIS_XX.wav
No.11754 path : /home/naokitakatani/datasets/CREMA-D/1087_DFA_FEA_XX.wav
No.11755 path : /home/naokitakatani/datasets/CREMA-D/1087_DFA_HAP_XX.wav
No.11756 path : /home/naokitakatani/datasets/CREMA-D/1087_DFA_NEU_XX.wav
No.11757 path : /home/naokitakatani/datasets/CREMA-D/1087_DFA_SAD_XX.wav
No.11758 path : /home/naokitakatani/datasets/CREMA-D/1087_IEO_ANG_HI.wav
No.11759 path : /home/naokitakatani/datasets/CREMA-D/1087_IEO_ANG_LO.wav
No.11760 path : /home/naokitakatani/datasets/CREMA-

No.11860 path : /home/naokitakatani/datasets/CREMA-D/1088_IOM_NEU_XX.wav
No.11861 path : /home/naokitakatani/datasets/CREMA-D/1088_IOM_SAD_XX.wav
No.11862 path : /home/naokitakatani/datasets/CREMA-D/1088_ITH_ANG_XX.wav
No.11863 path : /home/naokitakatani/datasets/CREMA-D/1088_ITH_DIS_XX.wav
No.11864 path : /home/naokitakatani/datasets/CREMA-D/1088_ITH_FEA_XX.wav
No.11865 path : /home/naokitakatani/datasets/CREMA-D/1088_ITH_HAP_XX.wav
No.11866 path : /home/naokitakatani/datasets/CREMA-D/1088_ITH_NEU_XX.wav
No.11867 path : /home/naokitakatani/datasets/CREMA-D/1088_ITH_SAD_XX.wav
No.11868 path : /home/naokitakatani/datasets/CREMA-D/1088_ITS_ANG_XX.wav
No.11869 path : /home/naokitakatani/datasets/CREMA-D/1088_ITS_DIS_XX.wav
No.11870 path : /home/naokitakatani/datasets/CREMA-D/1088_ITS_FEA_XX.wav
No.11871 path : /home/naokitakatani/datasets/CREMA-D/1088_ITS_HAP_XX.wav
No.11872 path : /home/naokitakatani/datasets/CREMA-D/1088_ITS_NEU_XX.wav
No.11873 path : /home/naokitakatani/datasets/CREMA-

No.11973 path : /home/naokitakatani/datasets/CREMA-D/1089_MTI_SAD_XX.wav
No.11974 path : /home/naokitakatani/datasets/CREMA-D/1089_TAI_ANG_XX.wav
No.11975 path : /home/naokitakatani/datasets/CREMA-D/1089_TAI_DIS_XX.wav
No.11976 path : /home/naokitakatani/datasets/CREMA-D/1089_TAI_FEA_XX.wav
No.11977 path : /home/naokitakatani/datasets/CREMA-D/1089_TAI_HAP_XX.wav
No.11978 path : /home/naokitakatani/datasets/CREMA-D/1089_TAI_NEU_XX.wav
No.11979 path : /home/naokitakatani/datasets/CREMA-D/1089_TAI_SAD_XX.wav
No.11980 path : /home/naokitakatani/datasets/CREMA-D/1089_TIE_ANG_XX.wav
No.11981 path : /home/naokitakatani/datasets/CREMA-D/1089_TIE_DIS_XX.wav
No.11982 path : /home/naokitakatani/datasets/CREMA-D/1089_TIE_FEA_XX.wav
No.11983 path : /home/naokitakatani/datasets/CREMA-D/1089_TIE_HAP_XX.wav
No.11984 path : /home/naokitakatani/datasets/CREMA-D/1089_TIE_NEU_XX.wav
No.11985 path : /home/naokitakatani/datasets/CREMA-D/1089_TIE_SAD_XX.wav
No.11986 path : /home/naokitakatani/datasets/CREMA-

No.12086 path : /home/naokitakatani/datasets/CREMA-D/1091_IEO_ANG_HI.wav
No.12087 path : /home/naokitakatani/datasets/CREMA-D/1091_IEO_ANG_LO.wav
No.12088 path : /home/naokitakatani/datasets/CREMA-D/1091_IEO_ANG_MD.wav
No.12089 path : /home/naokitakatani/datasets/CREMA-D/1091_IEO_DIS_HI.wav
No.12090 path : /home/naokitakatani/datasets/CREMA-D/1091_IEO_DIS_LO.wav
No.12091 path : /home/naokitakatani/datasets/CREMA-D/1091_IEO_DIS_MD.wav
No.12092 path : /home/naokitakatani/datasets/CREMA-D/1091_IEO_FEA_HI.wav
No.12093 path : /home/naokitakatani/datasets/CREMA-D/1091_IEO_FEA_LO.wav
No.12094 path : /home/naokitakatani/datasets/CREMA-D/1091_IEO_FEA_MD.wav
No.12095 path : /home/naokitakatani/datasets/CREMA-D/1091_IEO_HAP_HI.wav
No.12096 path : /home/naokitakatani/datasets/CREMA-D/1091_IEO_HAP_LO.wav
No.12097 path : /home/naokitakatani/datasets/CREMA-D/1091_IEO_HAP_MD.wav
No.12098 path : /home/naokitakatani/datasets/CREMA-D/1091_IEO_NEU_XX.wav
No.12099 path : /home/naokitakatani/datasets/CREMA-

In [13]:
preds = loaded_model.predict(features, 
                         batch_size=16, 
                         verbose=1)
#13の感情のなかから最も高い値(確率が高い感情)を選ぶ
preds=preds.argmax(axis=1)

761/761 [==============================] - 32s 42ms/step


In [14]:
with open("FilteredFeatures.pkl", "wb") as file:
    pickle.dump((features, preds), file)

In [15]:
with open("FilteredFeatures-preds.pkl", "wb") as file:
    pickle.dump(preds, file)

### 正解率を比較する

In [12]:
with open("nonFilterFeatures-preds.pkl", "rb") as file:
    preds_nF = pickle.load(file)

In [13]:
with open("FilteredFeatures-preds.pkl", "rb") as file:
    preds_Fd = pickle.load(file)

In [14]:
preds_sorce = np.empty(preds_nF.shape[0])
print(preds_nF.shape[0])
for index, label in enumerate(ref.labels):
    preds_sorce[index] = convlabel(label)

12162


In [15]:
split = [0, 480, 1919, 4720, 12162]
tcorrect = 0
for sp in range(len(split)-1):
    correct = 0
    for label, pred in zip(preds_sorce[split[sp]:split[sp+1]],preds_nF[split[sp]:split[sp+1]]):
        if label == pred:
            correct += 1
            tcorrect += 1
    accuracy = correct / preds_sorce[split[sp]:split[sp+1]].shape[0]
    print("accuracy:{}%".format(accuracy*100))
taccuracy = tcorrect / preds_sorce.shape[0]
print("##########\naccuracy:{}%".format(taccuracy*100))

accuracy:94.16666666666667%
accuracy:96.94232105628909%
accuracy:99.89289539450196%
accuracy:97.63504434291858%
##########
accuracy:97.93619470481829%


In [17]:
split = [0, 480, 1919, 4720, 12162]
tcorrect = 0
for sp in range(len(split)-1):
    correct = 0
    for label, pred in zip(preds_sorce[split[sp]:split[sp+1]],preds_Fd[split[sp]:split[sp+1]]):
        if label == pred:
            correct += 1
            tcorrect += 1
    accuracy = correct / preds_sorce[split[sp]:split[sp+1]].shape[0]
    print("accuracy:{}%".format(accuracy*100))
taccuracy = tcorrect / preds_sorce.shape[0]
print("##########\naccuracy:{}%".format(taccuracy*100))

accuracy:48.541666666666664%
accuracy:60.25017373175816%
accuracy:85.290967511603%
accuracy:75.27546358505778%
##########
accuracy:74.74921887847394%


In [ ]:
#やはりhaighpassをかけた方が感情認識結果が低い。

In [21]:
order = 2
bpfc1 = 1500
X, samplerate, samplewidth, nchannels = loadwav("/Users/naokitakatani/Documents/datasets/RAVDESS/Actor_07/03-01-04-01-02-02-07.wav")
ipd.display(ipd.Audio(X, rate=samplerate))
bpfc1 = 1500
b, a = signal.iirfilter(order, bpfc1,btype='highpass',rs = 60,rp = 1.0, ftype='ellip',output='ba',fs=samplerate)
Y = signal.lfilter(b, a, X)
ipd.display(ipd.Audio(Y, rate=samplerate))

In [22]:
order = 2
bpfc1 = 1500
X, samplerate, samplewidth, nchannels = loadwav("/Users/naokitakatani/Documents/datasets/SAVEE/JK_sa12.wav")
ipd.display(ipd.Audio(X, rate=samplerate))
bpfc1 = 1500
b, a = signal.iirfilter(order, bpfc1,btype='highpass',rs = 60,rp = 1.0, ftype='ellip',output='ba',fs=samplerate)
Y = signal.lfilter(b, a, X)
ipd.display(ipd.Audio(Y, rate=samplerate))